# Downloads and Imports

In [ ]:
!pip install pyspark
!pip install sparknlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 39 kB/s 
     |████████████████████████████████| 199 kB 38.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=3f5579a607aecc0201f20788f19bc557c78e04baff1eac4cb2318d1d892ca475
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 643 kB 28.4 MB/s 


In [ ]:
from pyspark import pandas as pd

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import dataframe
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

import sparknlp
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)


Spark NLP version:  4.2.2
Apache Spark version:  3.3.1


In [ ]:
'''!pip install sense2vec==1.0.3
!pip uninstall spacy
!pip install spacy==2.3.7'''

import spacy

print( spacy.__version__ )

3.4.2


In [ ]:
import json
import logging
import re

In [ ]:
persons_list = [( "alone", ["alone", "by myself"] ), ( "one friend", [" friend ", "buddy"] ), ( "multiple friends", [" friends", "buddies"] ), ( "family", ["family", "families", " son ", "daughter", " father", " mother", "mom", "dad", "brother", "sister", "bro ", "uncle", "aunt"] ), ( "partner", ["partner", "husband", "wife", "hubby", "boyfriend", "girlfriend", "gf", "bf", "GF", "BF", "SO"] )]
locations_list = [( "home", ["home", "house"] ), ( "school", ["school", "university"] ), ( "work", ["workplace", "job", "from work", "at work", "my work", "my office"] ), ( "bar", ["bar"] ), ( "supermarket", ["market", "store"] )]
data = \
{
    "content": "",
    "annotation": []
}

In [ ]:
def find_entity( post, entity ):
  index = -1
  starts = []
  ends = []
  while( 1 ):
    index = post.find( entity, index + 1 )
    if( index < 0 ):
      break
    starts.append( index if entity[0] != ' ' else index + 1 )
    ends.append( index + len( entity ) - 1 if entity[-1] != ' ' else index + len( entity ) - 2 )
  return starts, ends

In [ ]:
def add_to_JSON( post, entity, entity_type, starts, ends, data ):
  for i in range( len( starts ) ):
    
    dict_new = {
        "label": entity_type,
        "points": [{
            "start": starts[i],
            "end": ends[i],
            "text": entity
        }]
    }

    data['annotation'].append( dict_new )

In [ ]:
import random

In [ ]:
def get_training_data( posts ):
  for post in posts:
    data['content'] = post
    data['annotation'] = []
    for person in persons_list:
      entity, search_strings = person

      for search_string in search_strings:
        starts, ends = find_entity( post, search_string )
        # add_to_JSON( post, search_string, "Person: " + entity, starts, ends, data )
        # arr = ["sdf", "erg", "ery", "hfd", "eryer", "ngd", "wey", "erh"]
        # yikes = "multiple_friends" + arr[random.randint(0, 7)]
        add_to_JSON( post, search_string, "Person:" + entity, starts, ends, data )

    for location in locations_list:
      entity, search_strings = location

      for search_string in search_strings:
        starts, ends = find_entity( post, search_string )
        # add_to_JSON( post, search_string, "Location: " + entity, starts, ends, data )
        # arr = ["sdf", "erg", "ery", "hfd", "eryer", "ngd", "wey", "erh"]
        # yikes = "somewhere_really_far" + arr[random.randint(0, 7)]
        add_to_JSON( post, search_string, "Location:" + entity, starts, ends, data )
     
    json_object = json.dumps( data )

    with open( "/content/ejemplo.json", "a" ) as outfile:
      outfile.write( json_object )
      outfile.write( "\n" )

  # return convert_dataturks_to_spacy( "/content/ejemplo.json" )

In [ ]:
def convert_dataturks_to_conll( filepath, folder_path ):
  try:
    training_data = open( folder_path + ( "data.train" if folder_path[-1] == "/" else "/data.train" ), 'a' )
    docs = []
    with open( filepath, 'r' ) as f:
      docs = f.readlines()

    for doc in docs:
      char_counter = 0
      annotation_indices = {}

      data = json.loads( doc )
      text = data['content']
      for annot in data['annotation']:
        annotation_indices[annot['points'][0]['start']] = annot['label']#.replace( ": ", ":" )
      
      training_data.write( "-DOCSTART-\n\n" )      
      paragraphs = text.split( "\n" )
      for paragraph in paragraphs:
        training_data.write( "\n" )
        sentences = re.split( '\.|!|\?', paragraph )
        for sentence in sentences:
          words = sentence.split( " " )
          for word in words:
            training_data.write( word )
            if( char_counter in annotation_indices.keys() ):
              training_data.write( " " + annotation_indices[char_counter] )
            training_data.write( "\n" )
            char_counter += ( len( word ) + 1 )

          # training_data.write( "\n" )

  except Exception as e:
    logging.exception("Unable to process " + filepath + "\n" + "error = " + str(e))
    return None

In [ ]:
def convert_doccano_to_conll( filepath, folder_path ):
  try:
    test_data = open( folder_path + ( "data.testa" if folder_path[-1] == "/" else "/data.testa" ), 'a' )
    docs = []
    with open( filepath, 'r' ) as f:
      docs = f.readlines()

    for doc in docs:
      char_counter = 0
      annotation_indices = {}

      data = json.loads( doc )
      text = data['data']
      for annot in data['label']:
        annotation_indices[annot[0]] = annot[2].replace( ": ", ":" )#.replace( "e friend", "e_friend" )
      
      test_data.write( "-DOCSTART-\n\n" )      
      paragraphs = text.split( "\n" )
      for paragraph in paragraphs:
        test_data.write( "\n" )
        sentences = re.split( '\.|!|\?', paragraph )
        for sentence in sentences:
          words = sentence.split( " " )
          for word in words:
            test_data.write( word )
            if( char_counter in annotation_indices.keys() ):
              test_data.write( " " + annotation_indices[char_counter] )
            test_data.write( "\n" )
            char_counter += ( len( word ) + 1 )

          # training_data.write( "\n" )

  except Exception as e:
    logging.exception("Unable to process " + filepath + "\n" + "error = " + str(e))
    return None

In [ ]:
def convert_conll_to_csv( filepath, folder_path ):
  try:
    training_data = open( folder_path + ( "data.csv" if folder_path[-1] == "/" else "/data.csv" ), 'a' )
    lines = []
    with open( filepath, 'r' ) as f:
      lines = f.readlines()

    training_data.write( "Sentence #,Word,POS,Tag\n" )

    send_period = 0
    sentence_counter = 1

    training_data.write( "Sentence: " + str( sentence_counter ) )
    sentence_counter += 1

    for line in lines:
      data = line.split( " " )

      if( "," in data[0] or "\"" in data[0] ):
        data[0] = data[0].replace( "\"", "\"\"" )
        data[0] = "\"" + ( data[0][0:-1] if len( data ) == 1 else data[0] ) + "\"" + ( "\n" if len( data ) == 1 else "" )
         
      if len( data ) == 1: # no entity label
        if len( data[0][0:-1] ) == 0: # if blank line
          if send_period:
            training_data.write( ",.,.,O\n" )
            training_data.write( "Sentence: " + str( sentence_counter ) )
            sentence_counter += 1
            send_period = 0
        elif data[0][0] == "#": # if label line, we discard
          send_period = 0
        else: # if normal unlabeled word
          training_data.write( "," + data[0][0:-1] + ",.,O\n" )
          send_period = 1
      elif len( data ) == 2: # there is an entity label
        training_data.write( "," + data[0] + ",.," + data[1] )
        send_period = 1
      else:
        print( "yikes" )        

  except Exception as e:
    logging.exception("Unable to process " + filepath + "\n" + "error = " + str(e))
    return None

In [ ]:
convert_conll_to_csv( "/content/data.train", "/content/" )

In [ ]:
strinky = "hello,"
binky = strinky.split( " " )
( ( "\"" + data[0][0:-1] + "\"" ) if( "," in data[0][0:-1] ) else data[0][0:-1] )
trinky

'"hello,"'

In [ ]:
txt = "So. Real quick. Have found this group.\nHave been inspired to do it."

x = re.split('\.|!|\?', txt)

print(x)

for dude in x:
  guy = dude.split( "\n" )
  print( guy )

['So', ' Real quick', ' Have found this group', '']
['So']
['', 'Real', 'quick']
['', 'Have', 'found', 'this', 'group']
['']


In [ ]:
folder_path = "/content/"

training_path = folder_path + ( "data.train" if folder_path[-1] == "/" else "/data.train" )

training_path

'/content/data.train'

In [ ]:
stringy = "ugh bub"
fatso = stringy.replace( " ", "" ).replace( "b", "d" )
fatso

'ughdud'

# Hedditch Data Authentication

In [ ]:
import requests
import pandas as pd

In [ ]:
# personal use script, secret token
auth = requests.auth.HTTPBasicAuth( 'JNpGyLI-oJ_drb93Dykw9g', 'KgImJaXURcpJG4Ud9-HLBsj8UnnnRQ' )

data = {'grant_type': 'password',
        'username': 'shawwquille_aww_neal',
        'password': 'rampat11' }

headers = {'User-Agent': 'SquatBotTUM/0.0.1' }

res = requests.post( 'https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers )

token1 = res.json()['access_token']

headers = {**headers, **{'Authorization': f"bearer {token1}"}}

requests.get( 'https://oauth.reddit.com/api/v1/me', headers=headers )

<Response [200]>

In [ ]:
res = requests.get( "https://oauth.reddit.com/r/stopdrinking/new", headers=headers, params={'limit': '200'} )

print( res.json() )

{'kind': 'Listing', 'data': {'after': 't3_yo71f1', 'dist': 100, 'modhash': None, 'geo_filter': '', 'children': [{'kind': 't3', 'data': {'approved_at_utc': None, 'subreddit': 'stopdrinking', 'selftext': "I'm learning how to embroider and when I move at the end of this month into my new, bigger apartment, I'm going to set up my sewing machine and sew a quilt for my senior dog with themes from raising her. I think if I can keep myself busy in my downtime rather than drinking on the couch and watching TV... Maybe I can do this.", 'author_fullname': 't2_1714gh', 'saved': False, 'mod_reason_title': None, 'gilded': 0, 'clicked': False, 'title': '2 days without a drink. (today will be day 3)', 'link_flair_richtext': [], 'subreddit_name_prefixed': 'r/stopdrinking', 'hidden': False, 'pwls': 0, 'link_flair_css_class': None, 'downs': 0, 'top_awarded_type': None, 'hide_score': True, 'name': 't3_yonk8b', 'quarantine': False, 'link_flair_text_color': 'dark', 'upvote_ratio': 1.0, 'author_flair_backgro

In [ ]:
df = pd.DataFrame()

for post in res.json()['data']['children']:
  df = df.append({
      'subreddit': post['data']['subreddit'],
      'title': post['data']['title'],
      'selftext': post['data']['selftext'],
      'upvote_ratio': post['data']['upvote_ratio'],
      'ups': post['data']['ups'],
      'downs': post['data']['downs'],
      'score': post['data']['score']
  }, ignore_index=True )

In [ ]:
df.head()

,subreddit,title,selftext,upvote_ratio,ups,downs,score
0,stopdrinking,2 days without a drink. (today will be day 3),I'm learning how to embroider and when I move ...,1.00,2.0,0.0,2.0
1,stopdrinking,First landmark for me!,This is my first day 4 in a long time…. But th...,1.00,6.0,0.0,6.0
2,stopdrinking,Feeling Kinda Creaky Today,But at my age I come by it honorably. 16 days...,1.00,4.0,0.0,4.0
3,stopdrinking,Unbelievably Tired,"I’m trying to wake up on this Day 2, but I am ...",1.00,2.0,0.0,2.0
4,stopdrinking,1 month sober today,"Hello,\n\nI am Mats, Male, 28 years old and I ...",0.95,15.0,0.0,15.0


In [ ]:
persons_list = [( "alone", ["alone", "by myself"] ), ( "one_friend", [" friend ", "buddy"] ), ( "multiple_friends", [" friends", "buddies"] ), ( "family", ["family", "families", " son ", "daughter", " father", " mother", "mom", "dad", "brother", "sister", "bro ", "uncle", "aunt"] ), ( "partner", ["partner", "husband", "wife", "hubby", "boyfriend", "girlfriend", "gf", "bf", "GF", "BF", "SO"] )]
locations_list = [( "home", ["home", "house"] ), ( "school", ["school", "university"] ), ( "work", ["workplace", "job", "from work", "at work", "my work", "my office"] ), ( "bar", ["bar"] ), ( "supermarket", ["market", "store"] ), ( "airport", ["airport", "airplane"] )]

persons_list = [( "alone", ["alone", "by myself"] ), ( "one_friend", [" friend ", "buddy"] ), ( "multiple_friends", [" friends", "buddies"] ), ( "family", ["family", "families", " son ", "daughter", " father", " mother", "mom", "dad", "brother", "sister", "bro ", "uncle", "aunt"] ), ( "partner", ["partner", "husband", "wife", "hubby", "boyfriend", "girlfriend", "gf", "bf", "GF", "BF", "SO"] ), ( "stranger", ["stranger"] )]
# locations_list = [( "home", ["home", "house"] ), ( "school", ["school", "university"] ), ( "work", ["workplace", "job", "from work", "at work", "my work", "my office"] )]


data = \
{
    "content": "",
    "annotation": []
}

get_training_data( df['selftext'] )

In [ ]:
convert_dataturks_to_conll( "/content/ejemplo.json", "/content/" )

In [ ]:
from sparknlp.training import CoNLL
training_data = CoNLL( conllLabelIndex=1, posCol="disregard this column").readDataset(spark, '/content/data.train')
training_data.show()

+--------+--------------------+--------------------+--------------------+---------------------+--------------------+
|    text|            document|            sentence|               token|disregard this column|               label|
+--------+--------------------+--------------------+--------------------+---------------------+--------------------+
|  barely|[{document, 0, 5,...|[{document, 0, 5,...|[{token, 0, 5, ba...| [{pos, 0, 5, Loca...|[{named_entity, 0...|
|  moment|[{document, 0, 5,...|[{document, 0, 5,...|[{token, 0, 5, mo...| [{pos, 0, 5, Pers...|[{named_entity, 0...|
|  friend|[{document, 0, 5,...|[{document, 0, 5,...|[{token, 0, 5, fr...| [{pos, 0, 5, Pers...|[{named_entity, 0...|
|  friend|[{document, 0, 5,...|[{document, 0, 5,...|[{token, 0, 5, fr...| [{pos, 0, 5, Pers...|[{named_entity, 0...|
|    home|[{document, 0, 3,...|[{document, 0, 3,...|[{token, 0, 3, ho...| [{pos, 0, 3, Loca...|[{named_entity, 0...|
|   house|[{document, 0, 4,...|[{document, 0, 4,...|[{token, 0, 

In [ ]:
training_data_dos = CoNLL( conllLabelIndex=1, explodeSentences=False, posCol="disregard this column").readDataset(spark, '/content/data.train')


In [ ]:
training_data.selectExpr( "text", "token.result as tokens", "label.result as entity" ).show( 10, False )

+-------+---------+-------------------+
|text   |tokens   |entity             |
+-------+---------+-------------------+
|barely |[barely] |[Location:bar]     |
|moment |[moment] |[Person:family]    |
|friend |[friend] |[Person:one_friend]|
|friend |[friend] |[Person:one_friend]|
|home   |[home]   |[Location:home]    |
|house  |[house]  |[Location:home]    |
|husband|[husband]|[Person:partner]   |
|friend |[friend] |[Person:one_friend]|
|friend |[friend] |[Person:one_friend]|
|home   |[home]   |[Location:home]    |
+-------+---------+-------------------+
only showing top 10 rows



In [ ]:
training_data.show(100, False)

+-------------+----------------------------------------------------------+-------------------------------------------------------+-----------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text         |document                                                  |sentence                                               |token                                                                                    |disregard this column                                                                                                                        |label                                                                                                              

In [ ]:
bert = BertEmbeddings.pretrained('bert_base_cased', 'en') \
.setInputCols(["sentence",'token'])\
.setOutputCol("bert")\
.setCaseSensitive(False)
# .setPoolingLayer(0) # default 0

# credit for this section: https://towardsdatascience.com/named-entity-recognition-ner-with-bert-in-spark-nlp-874df20d1d77

bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]


In [ ]:
nerTagger = NerDLApproach()\
.setInputCols(["sentence", "token", "bert"])\
.setLabelColumn("label")\
.setOutputCol("ner")\
.setMaxEpochs(10)\
.setRandomSeed(0)\
.setVerbose(1)\
.setValidationSplit(0.2)\
.setEvaluationLogExtended(True)\
.setEnableOutputLogs(True)\
.setIncludeConfidence(True)\
.setTestDataset("/content/test_withEmbeds.parquet")

In [ ]:
convert_doccano_to_conll( "/content/admin.jsonl", "/content" )

test_data = CoNLL().readDataset(spark, '/content/data.testa')
test_data = bert.transform(test_data)
test_data.write.parquet("/content/test_withEmbeds.parquet")

In [ ]:
ner_pipeline = Pipeline(stages = [bert, nerTagger])
ner_model = ner_pipeline.fit(training_data)

In [ ]:
ner_model.stages[1].write().save('NER_bert_2')

In [ ]:
loaded_ner_model = NerDLModel.load("NER_bert_2")\
   .setInputCols(["sentence", "token", "bert"])\
   .setOutputCol("ner")

In [ ]:
document = DocumentAssembler().setInputCol( "text" ).setOutputCol( "document" )
sentence = SentenceDetector().setInputCols( ['document'] ).setOutputCol( 'sentence' )
token = Tokenizer().setInputCols( ['sentence'] ).setOutputCol( 'token' )

bert = BertEmbeddings.pretrained( 'bert_base_cased', 'en' )\
.setInputCols( ["sentence", "token"] )\
.setOutputCol( "bert" )\
.setCaseSensitive( False )

loaded_ner_model = NerDLModel.load( "NER_bert_2" )\
.setInputCols( ["sentence", "token", "bert"] )\
.setOutputCol( "ner" )

converter = NerConverter().setInputCols( ["document", "token", "ner"] ).setOutputCol( "ner_span" )

custom_ner_pipeline = Pipeline(
    stages=[
        document,
        sentence,
        token,
        bert,
        loaded_ner_model,
        converter
    ]
)

bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]


In [ ]:
import pyspark.sql.functions as F

def pyspark_displayer( df ):
  df.select( F.explode( F.arrays_zip( "ner_span.result", "ner_span.metadata" ) ).alias( "entities" ) )\
  .select( F.expr( "entities['chunk']" ).alias( "chunk" ),
          F.expr( "entities['entity']" ).alias( "entity" ) )\
          .show( truncate=False )

In [ ]:
with open( "/content/admin.jsonl", 'r' ) as f:
  docs = f.readlines()

# test_set_DF = spark.createDataFrame( [[]] )

for doc in docs:
  data = json.loads( doc )
  text = data['data']

  prediction_data = spark.createDataFrame( [[text]] ).toDF( "text" )
  # print( "prediction_data is a", type( prediction_data ) )

  prediction_model = custom_ner_pipeline.fit( prediction_data )

  predictions = prediction_model.transform( prediction_data )
  # print( "predictions is a", type( predictions ) )

  # test_set_DF.append( predictions )

  predictions.select( 'token.result', 'ner.result' ).show( truncate=False )
  # pyspark_displayer( predictions )

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
text = "Hi mom and dad"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = custom_ner_pipeline.fit(prediction_data)
preds = prediction_model.transform(prediction_data)
preds.show()
preds.select( 'token.result', 'ner.result' ).show( truncate=False )

+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|          text|            document|            sentence|               token|                bert|                 ner|            ner_span|
+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Hi mom and dad|[{document, 0, 13...|[{document, 0, 13...|[{token, 0, 1, Hi...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 13, H...|
+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

+-------------------+----------------------------------------------------------------------+
|result             |result                                                                |
+-------------------+----------------------------------------------------------------------+
|[Hi,

In [ ]:
pyspark_displayer( preds )

AnalysisException: ignored